In [127]:
from PIL import Image, ImageDraw

In [128]:
def draw_natural_line(draw, x1, y1, x2, y2):
    """
    Естественный алгоритм рисования прямой 
    """
    dx = x2 - x1
    dy = y2 - y1
    steps = max(abs(dx), abs(dy))
    x_inc = dx / float(steps)
    y_inc = dy / float(steps)
    x = x1
    y = y1
    for _ in range(steps + 1):
        draw.point((round(x), round(y)), fill="black")
        x += x_inc
        y += y_inc


In [129]:
def draw_bresenham_circle(draw, x0, y0, radius):
    """
    Алгоритм Брезенхама для рисования окружности 
    """
    x = 0
    y = radius
    p = 3 - 2 * radius
    while y >= x:
        draw.point((x0 + x, y0 + y), fill="black")
        draw.point((x0 + x, y0 - y), fill="black")
        draw.point((x0 - x, y0 + y), fill="black")
        draw.point((x0 - x, y0 - y), fill="black")
        draw.point((x0 + y, y0 + x), fill="black")
        draw.point((x0 + y, y0 - x), fill="black")
        draw.point((x0 - y, y0 + x), fill="black")
        draw.point((x0 - y, y0 - x), fill="black")
        if p <= 0:
            p += 4 * x + 6
        else:
            p += 4 * (x - y) + 10
            y -= 1
        x += 1


In [130]:
def draw_bezier_curve(draw, p0, p1, p2, p3, num_points=100):
    """
    Геометрический способ построения кривых Безье
    """
    for t in range(num_points + 1):
        t /= num_points
        x = int((1 - t) ** 3 * p0[0] + 3 * (1 - t) ** 2 * t * p1[0] + 3 * (1 - t) * t ** 2 * p2[0] + t ** 3 * p3[0])
        y = int((1 - t) ** 3 * p0[1] + 3 * (1 - t) ** 2 * t * p1[1] + 3 * (1 - t) * t ** 2 * p2[1] + t ** 3 * p3[1])
        draw.point((x, y), fill="black")


In [131]:
def flood_fill(image, x, y, target_color, replacement_color):
    """
    Короед
    """
    pixels = image.load()
    width, height = image.size

    if pixels[x, y] != target_color:
        return

    stack = [(x, y)]
    
    while stack:
        x, y = stack.pop()
        if pixels[x, y] == target_color:
            pixels[x, y] = replacement_color
            
            # Проверка границ перед добавлением соседей в стек
            if x > 0:
                stack.append((x - 1, y))
            if x < width - 1:
                stack.append((x + 1, y))
            if y > 0:
                stack.append((x, y - 1))
            if y < height - 1:
                stack.append((x, y + 1))

In [132]:
def modified_flood_fill(image, x, y, replacement_color):
    """
    Модифицированный рекурсивный алгоритм с "затравкой"
    """
    pixels = image.load()
    width, height = image.size
    stack = [(x, y)]
    target_color = pixels[x, y]
    while stack:
        x, y = stack.pop()
        # Закрашиваем текущую точку
        if pixels[x, y] == target_color:
            pixels[x, y] = replacement_color
            # Шаг 2: Закрашиваем максимально вправо и влево по строке
            x_left = x
            x_right = x
            # Закрашиваем вправо
            while x_right < width - 1 and pixels[x_right + 1, y] == target_color:
                x_right += 1
                pixels[x_right, y] = replacement_color
            # Закрашиваем влево
            while x_left > 0 and pixels[x_left - 1, y] == target_color:
                x_left -= 1
                pixels[x_left, y] = replacement_color
            # Шаг 3: Находим крайние правые пиксели в строке ниже
            if y + 1 < height:  # Если строка ниже существует
                for x_pos in range(x_left, x_right + 1):
                    if pixels[x_pos, y + 1] == target_color:
                        stack.append((x_pos, y + 1))
            # Шаг 4: Находим крайние правые пиксели в строке выше
            if y - 1 >= 0:  # Если строка выше существует
                for x_pos in range(x_left, x_right + 1):
                    if pixels[x_pos, y - 1] == target_color:
                        stack.append((x_pos, y - 1))


In [133]:
img_size = (200, 200)
house_img = Image.new("RGB", img_size, "white")
draw = ImageDraw.Draw(house_img)

# Рисуем стены
draw_natural_line(draw, 50, 50, 50, 150)  
draw_natural_line(draw, 50, 150, 150, 150)  
draw_natural_line(draw, 150, 150, 150, 50)  
draw_natural_line(draw, 150, 50, 50, 50)  

# Рисуем окна (окружности)
draw_bresenham_circle(draw, 75, 80, 5)  
draw_bresenham_circle(draw, 125, 80, 10)  

# Рисуем крышу с помощью кривой Безье
p0 = (50, 50)  # Левая вершина
p1 = (180, 10)  # Верхняя точка 0
p2 = (20, 10) # Верхняя точка 1
p3 = (150, 50)  # Правая вершина
draw_bezier_curve(draw, p0, p1, p2, p3, num_points=400)  # Крыша

# Закраска заднего фона с помощью алгоритма "короеда"
target1 = house_img.getpixel((10, 10))
replacement1 = (150, 150, 150)
flood_fill(house_img, 10, 10, target1, replacement1)

# Закраска крыши с помощью мод алг
modified_flood_fill(house_img, 100, 35, (150, 75, 0))

# Рисуем дверь
draw_natural_line(draw, 75, 150, 75, 100)  
draw_natural_line(draw, 75, 100, 125, 100)  
draw_natural_line(draw, 125, 100, 125, 150)
draw_bresenham_circle(draw, 85, 130, 5)
modified_flood_fill(house_img, 115, 130, (300, 75, 100))
target2 = house_img.getpixel((85, 130))
replacement2 = (0, 0, 200)
flood_fill(house_img, 85, 130, target2, replacement2)

house_img.show()